In [1]:
!pip install pulp pandas openpyxl

In [2]:
import pulp
import pandas as pd

# Load Input File

In [3]:
# Input file path
excel_file = 'ngs_fairness_solver_input_tmpl.xlsx'  # Change this to your actual file name

# Load period demands
periods_df = pd.read_excel(excel_file, sheet_name='Periods')
# columns: ['period', 'SA', 'A']

# Load target proportions
targets_df = pd.read_excel(excel_file, sheet_name='Target Proportions')
# columns: ['entity_type', 'entity_code', 'target_proportions']

# Extract sets
periods = periods_df['period'].tolist()
ranks = ['SA', 'A']
watches = targets_df[targets_df['entity_type'] == 'watch']['entity_code'].tolist()
divisions = targets_df[targets_df['entity_type'] == 'division']['entity_code'].tolist()

# Target proportions
watch_targets = targets_df[targets_df['entity_type'] == 'watch'].set_index('entity_code')['target_proportion'].to_dict()
division_targets = targets_df[targets_df['entity_type'] == 'division'].set_index('entity_code')['target_proportion'].to_dict()

# Set Tolerance Level

In [4]:
# Tolerance for overall ratio constraints (e.g., 2%)
TOLERANCE = 0.00

# Build Model

In [5]:
model = pulp.LpProblem("Fair_Sparse_Deployment", pulp.LpMinimize)

# Decision variables: x[rank, watch, division, period]
x = pulp.LpVariable.dicts(
    "x",
    ((rank, w, d, p) for rank in ranks for w in watches for d in divisions for p in periods),
    lowBound=0,
    cat='Integer'
)

# Max concentration per period and rank (for sparsity)
z = pulp.LpVariable.dicts(
    "z",
    ((rank, p) for rank in ranks for p in periods),
    lowBound=0,
    cat='Integer'
)

# Slack variables for intra-division watch ratios (across all periods)
slack_divwatch_pos = pulp.LpVariable.dicts(
    "slack_divwatch_pos",
    ((rank, d, w) for rank in ranks for d in divisions for w in watches),
    lowBound=0,
    cat='Continuous'
)

slack_divwatch_neg = pulp.LpVariable.dicts(
    "slack_divwatch_neg",
    ((rank, d, w) for rank in ranks for d in divisions for w in watches),
    lowBound=0,
    cat='Continuous'
)


# Set Constraints

In [6]:
# 1. Period demand constraints
for idx, p in enumerate(periods):
    for rank in ranks:
        period_demand = periods_df.loc[idx, rank]
        model += (
            pulp.lpSum(x[rank, w, d, p] for w in watches for d in divisions) == period_demand,
            f"Demand_{rank}_{p}"
        )

# 2. Overall watch ratio constraints (with tolerance)
for rank in ranks:
    total_rank = periods_df[rank].sum()
    for w in watches:
        target = watch_targets[w] * total_rank
        tol = max(1, int(round(TOLERANCE * total_rank)))
        actual = pulp.lpSum(x[rank, w, d, p] for d in divisions for p in periods)
        model += (actual >= target - tol), f"WatchLow_{rank}_{w}"
        model += (actual <= target + tol), f"WatchHigh_{rank}_{w}"

# 3. Overall division ratio constraints (with tolerance)
for rank in ranks:
    total_rank = periods_df[rank].sum()
    for d in divisions:
        target = division_targets[d] * total_rank
        tol = max(1, int(round(TOLERANCE * total_rank)))
        actual = pulp.lpSum(x[rank, w, d, p] for w in watches for p in periods)
        model += (actual >= target - tol), f"DivLow_{rank}_{d}"
        model += (actual <= target + tol), f"DivHigh_{rank}_{d}"

# 4. Sparsity (soft) constraint: link x to z
for rank in ranks:
    for p in periods:
        for w in watches:
            for d in divisions:
                model += x[rank, w, d, p] <= z[rank, p], f"Sparse_{rank}_{w}_{d}_{p}"

# 5. Intra-Division Watch Ratios
for rank in ranks:
    total_rank = periods_df[rank].sum()
    for d in divisions:
        # Total deployments for this division (already constrained by division_targets)
        total_div = division_targets[d] * total_rank
        
        for w in watches:
            # Target: watch_targets[w] * total_div
            target = watch_targets[w] * total_div
            # Actual: sum of this watch's deployments in this division across all periods
            actual = pulp.lpSum(x[rank, w, d, p] for p in periods)
            
            # Soft constraint with slack
            model += (actual - target <= slack_divwatch_pos[rank, d, w])
            model += (target - actual <= slack_divwatch_neg[rank, d, w])


In [7]:
# objective: minimize z + slack deviations for intra-division watch ratios
model += (
    pulp.lpSum(z[rank, p] for rank in ranks for p in periods) +
    0.1 * pulp.lpSum(slack_divwatch_pos[rank, d, w] + slack_divwatch_neg[rank, d, w] 
                      for rank in ranks for d in divisions for w in watches),
    "Total_Objective"
)

# Solve

In [8]:
solver = pulp.PULP_CBC_CMD(msg=True)
result_status = model.solve(solver)
print("Solver status:", pulp.LpStatus[model.status])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/88004977da564137acab8d1093ebb87a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/88004977da564137acab8d1093ebb87a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 457 COLUMNS
At line 3994 RHS
At line 4447 BOUNDS
At line 4760 ENDATA
Problem MODEL has 452 rows, 412 columns and 2800 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 14.4256 - 0.01 seconds
Cgl0003I 0 fixed, 12 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 432 rows, 412 columns (312 integer (0 of which binary)) and 2200 elements
Cbc0012I Integer solution of 20.947732 found by DiveCoefficient after 1285 iterations and 0 nodes (0.27 seconds)
Cbc0031I 128 added rows had average density of 17

# Display Results

In [9]:
ordered_divisions = ['HOS', 'KWL', 'KCE', 'NTE', 'NTW']
ordered_watches = ['A', 'B', 'C', 'D', 'E']

In [12]:
if pulp.LpStatus[model.status] == 'Optimal':
    # Gather results
    deployment_data = []
    for rank in ranks:
        for p in periods:
            for w in watches:
                for d in divisions:
                    val = int(pulp.value(x[rank, w, d, p]))
                    deployment_data.append({
                        'period': p,
                        'rank': rank,
                        'watch': w,
                        'division': d,
                        'count': val
                    })
    results_df = pd.DataFrame(deployment_data)

    # Print deployment matrices for each period and rank
    for rank in ranks:
        print(f"\n{'='*30}\n{rank} DEPLOYMENTS\n{'='*30}")
        rank_df = results_df[results_df['rank'] == rank]
        for p in periods:
            print(f"\nPeriod {p}:")
            period_df = rank_df[rank_df['period'] == p]
            # Build the grid with desired order
            grid = pd.DataFrame(0, index=ordered_divisions, columns=ordered_watches)
            for _, row in period_df.iterrows():
                grid.at[row['division'], row['watch']] = row['count']
            grid.loc['TOTAL'] = grid.sum()
            grid['TOTAL'] = grid.sum(axis=1)
            print(grid)


    # Print overall actual vs target proportions
    for rank in ranks:
        print(f"\n{'='*30}\n{rank} OVERALL PROPORTIONS\n{'='*30}")
        rank_df = results_df[results_df['rank'] == rank]
        total = rank_df['count'].sum()
        print("Watch proportions:")
        watch_actual = rank_df.groupby('watch')['count'].sum() / total
        for w in watches:
            print(f"  {w}: Actual={watch_actual.get(w,0):.2%}  Target={watch_targets[w]:.2%}  Delta={watch_actual.get(w,0)-watch_targets[w]:+.2%}")
        print("Division proportions:")
        div_actual = rank_df.groupby('division')['count'].sum() / total
        for d in divisions:
            print(f"  {d}: Actual={div_actual.get(d,0):.2%}  Target={division_targets[d]:.2%}  Delta={div_actual.get(d,0)-division_targets[d]:+.2%}")

    print("\n" + "="*50)
    print("INTRA-DIVISION WATCH RATIO SUMMARY")
    print("="*50)
    
    for rank in ranks:
        print(f"\n{'='*30}\n{rank} INTRA-DIVISION WATCH RATIOS\n{'='*30}")
        rank_df = results_df[results_df['rank'] == rank]
        for d in divisions:
            div_df = rank_df[rank_df['division'] == d]
            total_in_div = div_df['count'].sum()
            print(f"\nDivision {d} (Total assigned: {total_in_div}):")
            summary = []
            for w in watches:
                actual = div_df[div_df['watch'] == w]['count'].sum()
                actual_prop = actual / total_in_div if total_in_div > 0 else 0
                target_prop = watch_targets[w]
                delta = actual_prop - target_prop
                summary.append({
                    'Watch': w,
                    'Actual': f"{actual_prop:.2%}",
                    'Target': f"{target_prop:.2%}",
                    'Delta': f"{delta:+.2%}"
                })
            summary_df = pd.DataFrame(summary).set_index('Watch')
            print(summary_df)

else:
    print("No optimal solution found. Try increasing TOLERANCE.")



SA DEPLOYMENTS

Period G1:
       A  B  C  D  E  TOTAL
HOS    0  0  0  1  1      2
KWL    1  1  0  0  0      2
KCE    0  0  0  0  1      1
NTE    0  0  0  1  0      1
NTW    0  0  0  0  0      0
TOTAL  1  1  0  2  2      6

Period G2:
       A  B  C  D  E  TOTAL
HOS    1  0  0  1  0      2
KWL    0  0  1  1  1      3
KCE    1  1  1  1  0      4
NTE    1  1  1  0  1      4
NTW    1  0  0  0  0      1
TOTAL  4  2  3  3  2     14

Period G3:
       A  B  C  D  E  TOTAL
HOS    2  2  2  1  2      9
KWL    2  2  1  2  1      8
KCE    1  2  2  2  2      9
NTE    1  0  2  2  1      6
NTW    0  2  2  2  2      8
TOTAL  6  8  9  9  8     40

Period G4:
       A  B  C  D  E  TOTAL
HOS    0  0  0  1  0      1
KWL    1  0  1  1  1      4
KCE    1  1  1  0  0      3
NTE    0  1  0  1  1      3
NTW    1  1  1  1  1      5
TOTAL  3  3  3  4  3     16

Period S1:
       A  B  C  D  E  TOTAL
HOS    0  0  0  0  0      0
KWL    0  0  0  0  0      0
KCE    0  0  0  0  0      0
NTE    1  0  0  0  0      1


# Save Result to xlsx

In [11]:
output_excel = "deployment_report.xlsx"
with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    for p in periods:
        for rank in ranks:
            rank_df = results_df[results_df['rank'] == rank]
            period_df = rank_df[rank_df['period'] == p]
            grid = pd.DataFrame(0, index=ordered_divisions, columns=ordered_watches)
            for _, row in period_df.iterrows():
                grid.at[row['division'], row['watch']] = row['count']
            grid.loc['TOTAL'] = grid.sum()
            grid['TOTAL'] = grid.sum(axis=1)
            sheet_name = f"{rank}_{p}"
            grid.to_excel(writer, sheet_name=sheet_name)
        # Proportion tables
        total = rank_df['count'].sum()
        watch_actual = rank_df.groupby('watch')['count'].sum() / total
        watch_table = pd.DataFrame({
            'Actual': watch_actual,
            'Target': pd.Series(watch_targets),
            'Delta': watch_actual - pd.Series(watch_targets)
        })
        watch_table.index.name = 'Watch'
        watch_table = watch_table.apply(lambda col: col.map(lambda x: f"{x:.2%}" if isinstance(x, float) else x))
        watch_table.to_excel(writer, sheet_name=f"{rank}_Watch_Proportions")
        div_actual = rank_df.groupby('division')['count'].sum() / total
        div_table = pd.DataFrame({
            'Actual': div_actual,
            'Target': pd.Series(division_targets),
            'Delta': div_actual - pd.Series(division_targets)
        })
        div_table.index.name = 'Division'
        div_table = div_table.apply(lambda col: col.map(lambda x: f"{x:.2%}" if isinstance(x, float) else x))
        div_table.to_excel(writer, sheet_name=f"{rank}_Division_Proportions")

print(f"\nAll deployment matrices and proportion tables exported to {output_excel}")



All deployment matrices and proportion tables exported to deployment_report.xlsx
